In [1]:
#%run <path of the notebook that has to be run>

In [54]:
#import csv
from cobra.io import read_sbml_model, write_sbml_model
#from copy import copy
#from dotenv import find_dotenv
#import os
#from os.path import dirname

In [55]:
model = read_sbml_model('model_yeast8_rhb.xml')

In [56]:
print(model.objective)

Maximize
1.0*GROWTH - 1.0*GROWTH_reverse_c845e


### Function

In [72]:
def test_knockout(model, gene_id):
    """input: ID of the gene of interest, as type string.
    """
    with model as test_mutant:
        model.objective = model.reactions.rHb
        old_rhb_production = model.optimize().objective_value / (-1*model.reactions.EX_glc__D_e.flux)
    
    with model as test_mutant:
        old_rate = test_mutant.slim_optimize()
        
        gene = test_mutant.genes.get_by_id(gene_id)
        gene.knock_out() # gene to knock out
        new_rate = test_mutant.slim_optimize()
        rate_change = (new_rate - old_rate)
        print("The new growth rate is: "+str(new_rate)[:6]+". This is a change of "+str(rate_change)[:6]+" from the old growth rate, or a change of "+str(rate_change/old_rate*100)[:5]+"%.")
        
        test_mutant.objective = test_mutant.reactions.rHb
        new_rhb_production = test_mutant.optimize().objective_value / (-1*test_mutant.reactions.EX_glc__D_e.flux)
        
        print("Before the knockout, the theoretical maximum yield of rHb on glucose [mmol-rHb / mmol-glucose] was: "+ str(old_rhb_production)[0:]+" , now the yield is: "+str(new_rhb_production)[0:])

Knocking out a single gene

## Notes for Knockouts

##### Up-regulation

HEM2	YGL040C,
HEM3	YDL205C,
HEM13	YDR044W,
HEM15	YOR176W

##### Knockout

HAP1	YLR256W

##### Downregulation
YFH1	YDL120W

##### Maybe interesting
CCC1	YLR220W	Transport af heme - into vacule


PUG1	YER185W	Export af heme - ud af cellen

##### Knockout

YPL172C knockout

YAL039C knockout

#### Let's see whether COX10 is essential.

In [58]:
model.genes.YPL172C.name

'COX10'

In [71]:
test_knockout(model, 'YPL172C')

The new growth rate is: 0.0837. This is a change of 0.0 from the old growth rate, or a change of 0.0%.
Before the knockout, the theoretical maximum yield of rHb on glucose [mmol-rHb / mmol-glucose] was: 0.001595 , now the yield is: 0.001595


#### Now let's try for CYC3.

In [60]:
model.genes.YAL039C.name

'CYC3'

In [61]:
test_knockout(model, 'YAL039C')

The new growth rate is: 0.0837. This is a change of 0.0 from the old growth rate, or a change of 0.0%.
Before the knockout, the theoretical maximum yield of rHb on glucose [mmol-rHb / mmol-glucose] was: 0.0015 , now the yield is: 0.0015


#### Now let's try for CYC3.

In [62]:
model.genes.YDL120W.name

'YFH1'

In [73]:
test_knockout(model, 'YDL120W')

The new growth rate is: 0.0837. This is a change of 0.0 from the old growth rate, or a change of 0.0%.
Before the knockout, the theoretical maximum yield of rHb on glucose [mmol-rHb / mmol-glucose] was: 0.0015954490716580578 , now the yield is: 0.0015954490716580667


In [64]:
model.medium

{'EX_nh4_e': 1000.0,
 'EX_glc__D_e': 1.0,
 'EX_h_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0}

Maybe this is because iron(II) is not a limiting factor in the medium.

In [65]:
def test_knockout_iron(model, gene_id):
    """input: ID of the gene of interest, as type string.
    """
    with model as test_mutant:
        model.objective = model.reactions.rHb
        old_rhb_production = model.optimize().objective_value / (-1*model.reactions.EX_glc__D_e.flux)
        
    medium = model.medium
    with model as test_mutant:
        
        medium['EX_glc__D_e'] = 6
        #medium['EX_fe2_e'] = 100
        #medium['EX_o2_e'] = 100
        model.medium = medium
        solution = model.optimize()
        old_rate = test_mutant.slim_optimize()
        
        gene = test_mutant.genes.get_by_id(gene_id)
        gene.knock_out() # gene to knock out
        new_rate = test_mutant.slim_optimize()
        rate_change = (new_rate - old_rate)
        print("The new growth rate is: "+str(new_rate)[:6]+". This is a change of "
              +str(rate_change)[:6]+" from the old growth rate, or a change of "+str(rate_change/old_rate*100)[:5]+"%.")
        
        test_mutant.objective = test_mutant.reactions.rHb
        new_rhb_production = test_mutant.optimize().objective_value / (-1*test_mutant.reactions.EX_glc__D_e.flux)
        
        print("Before the knockout, the theoretical maximum yield of rHb on glucose [mmol-rHb / mmol-glucose] was: "+
              str(old_rhb_production)[:7]+" , now the yield is: "+str(new_rhb_production)[:7])

In [66]:
test_knockout_iron(model, 'YDL120W')

The new growth rate is: 0.5170. This is a change of 0.0 from the old growth rate, or a change of 0.0%.
Before the knockout, the theoretical maximum yield of rHb on glucose [mmol-rHb / mmol-glucose] was: 0.00159 , now the yield is: 0.00163


In [67]:
model.medium

{'EX_nh4_e': 1000.0,
 'EX_glc__D_e': 1.0,
 'EX_h_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0}

## FBA

In [13]:
#from dfba import DfbaModel, ExchangeFlux, KineticVariable

ModuleNotFoundError: No module named 'dfba'

TypeError: 'Objective' object is not callable